In [1]:
import kfp
from kfp import dsl
from kfp.components import func_to_container_op, InputPath, OutputPath

kfp.__version__

'1.8.19'

In [2]:
import os

with open(os.environ['KF_PIPELINES_SA_TOKEN_PATH'], "r") as f:
    TOKEN = f.read()

In [3]:
endpoint = 'http://ml-pipeline.kubeflow:8888'
kfp_client = kfp.Client(host=endpoint, existing_token=TOKEN)

In [4]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms, utils

torch.__version__

'1.8.1+cpu'

In [5]:
from minio import Minio

client = Minio(
    'minio.kubeflow:9000',
    access_key='minik8s',
    secret_key='minik8sio',
    secure=False,
)
client.bucket_exists("mnist")

True

In [6]:
from kubernetes import client as k8s_client
from functools import partial


torch_func_to_container_op = partial(
    func_to_container_op,
    base_image='bitnami/pytorch:1.8.1',
)

In [11]:
@torch_func_to_container_op
def load_dataset(mount_path: str):
    from torchvision import datasets, transforms
    
    transform = transforms.Compose([
        transforms.ToTensor(), # 轉為 Tensor
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)), # 灰階轉為 RGB
    ])
    mnist_train = datasets.MNIST(
        root=mount_path,       # 資料放置路徑
        train=True,          # 訓練資料集
        download=True,       # 自動下載
        transform=transform  # 轉換函數
    )


@dsl.pipeline(
   name='Test load dataset Pipeline',
)
def test_dataset_pipeline(mount_path: str):
    from kubernetes import client as k8s_client
    data_loading_task = (
        load_dataset(mount_path)
        .add_volume(k8s_client.V1Volume(name='torch-mnist-datavol'))
        .add_volume_mount(
            k8s_client.V1VolumeMount(
                mount_path=mount_path,
                name='torch-mnist-datavol',
            )
        )
    )

In [12]:
kfp.compiler.Compiler().compile(test_dataset_pipeline, 'test-dataset.yaml')

In [14]:
transform = transforms.Compose([
    transforms.ToTensor(), # 轉為 Tensor
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)), # 灰階轉為 RGB
])
mnist_train = datasets.MNIST(
    root='./data',       # 資料放置路徑
    train=True,          # 訓練資料集
    download=True,       # 自動下載
    transform=transform  # 轉換函數
)